# Abstractive Text Summarization Using Pegasus
_By: Ling Li Ya_

References:
1. [Exploring Pegasus - A New Text Summarization NLP Model](https://signal.onepointltd.com/post/102ghb9/exploring-pegasus-a-new-text-summarization-nlp-model)
2. [Notebook referred to prepare this notebook](https://colab.research.google.com/drive/1-zq8AJktuC3gQAHTuSiiZ_qvDl4wK7rq#scrollTo=S3PYeeGuda0m)

## 1. Install and Import Dependencies

In [17]:
!pip install sentencepiece

In [18]:
!pip install transformers

In [19]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, pipeline
from bs4 import BeautifulSoup
import torch

## 2. Create Model

In [20]:
# model_name = 'google/pegasus-xsum'
# torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [21]:
model_name = 'google/pegasus-xsum'
torch_device = 'cuda'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [22]:
src_text = [
    """Rococo (/rəˈkoʊkoʊ/, also US: /ˌroʊkəˈkoʊ/), less commonly Roccoco or Late Baroque, is an exceptionally ornamental and theatrical style of architecture, art and decoration which combines asymmetry, scrolling curves, gilding, white and pastel colors, sculpted molding, and trompe-l'œil frescoes to create surprise and the illusion of motion and drama. It is often described as the final expression of the Baroque movement.[1] The Rococo style began in France in the 1730s as a reaction against the more formal and geometric Style Louis XIV. It was known as the style rocaille, or rocaille style.[2] It soon spread to other parts of Europe, particularly northern Italy, Austria, southern Germany, Central Europe and Russia.[3] It also came to influence the other arts, particularly sculpture, furniture, silverware, glassware, painting, music, and theatre.[4] Although originally a secular style primarily used for interiors of private residences the Rococo had a spiritual aspect to it which led to its widespread use in church interiors, particularly in Central Europe, Portugal, and South America.[5]""", """The word rococo was first used as a humorous variation of the word rocaille.[6][7] Rocaille was originally a method of decoration, using pebbles, seashells and cement, which was often used to decorate grottoes and fountains since the Renaissance.[8][9] In the late 17th and early 18th century rocaille became the term for a kind of decorative motif or ornament that appeared in the late Style Louis XIV, in the form of a seashell interlaced with acanthus leaves. In 1736 the designer and jeweler Jean Mondon published the Premier Livre de forme rocquaille et cartel, a collection of designs for ornaments of furniture and interior decoration. It was the first appearance in print of the term "rocaille" to designate the style.[10] The carved or molded seashell motif was combined with palm leaves or twisting vines to decorate doorways, furniture, wall panels and other architectural elements.[11]"""
]

In [23]:
# batch = tokenizer.prepare_seq2seq_batch(src_text, truncation=True, padding='longest').to(torch_device)
# translated = model.generate(**batch)
# tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
# print(tgt_text)

In [30]:
summary = pipeline('summarization', model=model, tokenizer=tokenizer)
summary(src_text[0], min_length=40, max_length=60)

[{'summary_text': 'The style is often described as the final expression of the Baroque movement and it is often described as the final expression of the Rococo style.'}]